In [2]:
import cudf as pd
import cupy as np

In [3]:
import cuml

from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler
from cuml.metrics import r2_score, mean_absolute_error, mean_squared_error
from cuml.linear_model import LinearRegression



In [4]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [5]:
train.shape

In [6]:
test .shape

In [7]:
train.info()

In [9]:
#now I am making a new DF in which I will not use object Dtype
temp=[]
for val in train.columns:
    if train[val].dtype in ['int64', 'float64', 'int32', 'float32']:
        temp.append(val)
        
df = train[temp]

In [10]:
df.info()

In [11]:
df.isnull().sum()

In [19]:
#filling the mode value in place of null values
df.fillna(df.mode().iloc[0], inplace=True)

In [20]:
#rechecking null
df.isnull().sum()

In [33]:
X, Y = df.iloc[:, 1:-1], df.iloc[:,-1]
print(X.shape, Y.shape)

In [34]:
skewed_feauture = [col for col in X.columns if abs(X[col].skew())>0.5]

In [36]:
#normalising skewed features
for feature in skewed_feauture:
    X[feature] = np.log1p(X[feature])
    


In [40]:
from cuml.preprocessing import StandardScaler
ss = StandardScaler()
X_sc = ss.fit_transform(X)

In [42]:
from cuml.preprocessing import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.1, random_state=3)

In [66]:
# function for all models to return [r2_score, mean_abs_error, mean_sq_error]
def metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_t = y_test.astype('float64')
    r2, mae, mse = r2_score(y_t, y_pred), mean_absolute_error(y_t, y_pred), mean_squared_error(y_t, y_pred)
    print('R2:', r2)
    print('MAE:', mae)    
    print('MSE:', mse)
    return [r2, mae, mse]

In [67]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
models=[]
new=[]

In [68]:
for algo in algorithm:
    lr = LinearRegression(algorithm=algo)
    %time lr.fit(X_train, y_train)
    models.append(lr)
    new.append(metrics(lr, X_test, y_test))

In [69]:
linear_stat = pd.DataFrame(new, columns=['R2', 'MAE', 'MSE'], index=algorithm)

In [70]:
#Displaying the comparison
linear_stat